In [73]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import transformers

In [74]:
####################
# NOTE: this notebook assumes the brain tumor data is in a directory at the root
# of this repository called 'BrainTumorData'.
#
# To make this work:
# 1 - go to https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri
# 2 - download the .zip file to this directory
# 3 - unzip the archive
# 4 - rename the expanded directory to BrainTumorData


data_directory = os.path.join(os.getcwd(), 'BrainTumorData')
train_dir = os.path.join(data_directory, 'Training')
test_dir = os.path.join(data_directory, 'Testing')

train_imgs = []
train_labels = []

test_imgs = []
test_labels = []


# get labels from the file structure
for label in os.listdir(train_dir):
    for img in os.listdir(os.path.join(train_dir, label)):
        train_imgs.append (img) 
        train_labels.append(label)
        
for label in os.listdir(train_dir):
    for img in os.listdir(os.path.join(train_dir, label)):
        train_imgs.append (img) 
        train_labels.append(label)


In [75]:
# dataloaders only support tensors, np arrays, lists, dicts, and numbers
# define this function to cast PIL images to tensors
transform_fn = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  
    transforms.Normalize([0.5,0.5,0.5], 
                        [0.5,0.5,0.5])
])

# create a validation set from the test sset
x_train, x_val, y_train, y_val = train_test_split(np.array(train_imgs), np.array(train_labels), test_size=0.1)

# create dataloaders
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_dir, transform=transform_fn),
    batch_size=64,
    shuffle=True,
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_dir, transform=transform_fn),
    batch_size=32,
    shuffle=False,
)

In [76]:
# get class names
class_names = [label for label in os.listdir(train_dir)]

# support GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [77]:
# define the model and send it to the device
model = models.resnet18().to(device)

In [87]:
# some deep learning overhead to help the model learn better
# can tweak these if you want but don't need to
warmup_steps = 5000 # delays using learning rate
num_epochs = 10 # arbitrary for now
learning_rate = 5e-3 # intial LR used after warmup
weight_decay = 0.0 # throttles optimizer -- 0 for now

num_update_steps_per_epoch = len(train_loader)
max_train_steps = num_epochs * num_update_steps_per_epoch

optimizer = torch.optim.AdamW(
    model.parameters(), lr=learning_rate, weight_decay=weight_decay,
)

lr_scheduler = transformers.get_scheduler(
    name='linear',
    optimizer=optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=max_train_steps,
)

In [ ]:
def evaluate_model(model, test_loader, device):
    # set the model to evaluation mode
    model.eval()
    for batch in tqdm(dataloader, desc="Evaluation"):
        # do some math
        # return some numbers
        pass
        # Mike TODO

In [91]:
# training loop

for epoch in tqdm(range(num_epochs), desc='training epochs'):
    # set the model to train mode
    model.train()
    
    for images, labels in tqdm(train_loader, desc='train batches'):
        images = images.to(device)
        labels = labels.to(device)
                
        preds = model(images).to(device)
        
        loss = F.cross_entropy(preds,labels)
        
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        
    # evaluate at the end of each epoch
    # Mike TODO


# save the model checkpoint after training
# in case we want to use it again for class demo
output_dir = 'output'
try:
    os.mkdir(output_dir)
except:
    # we don't care if it already exists... move on
    pass

model.save_pretrained(output_dir)

batches:   0%|                                                                                                                                                      | 0/45 [00:00<?, ?it/s]

tensor(7.0976, grad_fn=<NllLossBackward0>)



batches:   2%|███▏                                                                                                                                          | 1/45 [00:05<04:09,  5.66s/it]

tensor(7.0643, grad_fn=<NllLossBackward0>)



batches:   4%|██████▎                                                                                                                                       | 2/45 [00:10<03:50,  5.36s/it]

...

tensor(5.8625, grad_fn=<NllLossBackward0>)



batches:  51%|████████████████████████████████████████████████████████████████████████                                                                     | 23/45 [02:04<02:00,  5.47s/it]

tensor(5.6643, grad_fn=<NllLossBackward0>)


epochs...:   0%|                                                                                                                                                    | 0/10 [02:10<?, ?it/s]


KeyboardInterrupt: 